In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.stem import LancasterStemmer
from wordcloud import STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC,LinearSVC
from sklearn import metrics




data=pd.read_csv('stocktwits_xom.csv')

data=data[data.label != 0]
data.label[data.label==-1]=0

# print(data.head())

stop_words = set(STOPWORDS)
def trimming(s):
    lancaster = LancasterStemmer()
    #remove http
    s = re.sub(r'http\S+', '', s)
    #remove $ hashtag
    s = re.sub(r'\$(\w+)','',s)
    #remove non alphabetic
    s = re.sub('([^\s\w]|_)+', '', s)
    #remove numbers
    s = re.sub(r'\d+','',s)
    return s


data['message'] = data['message'].apply(trimming)

stemmer=LancasterStemmer()



def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text)]
    # stems = [stemmer.stem(item) for item in tokens]
    # s=[]
    # for word in tokens:
    #     if word not in stop_words:
    #         s.append(stemmer.stem(word))
    return tokens




max_len = 20

x=data['message']
y=data['label']
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.33,random_state=42)




text_clf = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(train_x, train_y)



predictions = text_clf.predict(test_x)

print(metrics.confusion_matrix(test_y.values,predictions))

print(metrics.classification_report(test_y.values,predictions))

print(metrics.accuracy_score(test_y.values,predictions))

[[ 332  372]
 [ 143 2317]]
              precision    recall  f1-score   support

           0       0.70      0.47      0.56       704
           1       0.86      0.94      0.90      2460

   micro avg       0.84      0.84      0.84      3164
   macro avg       0.78      0.71      0.73      3164
weighted avg       0.83      0.84      0.83      3164

0.8372313527180784
